## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import all_estimators
from sklearn.base import RegressorMixin

## Define the files to be loaded

In [2]:
# Participant number whose data should be loaded
participant_number = 3

# The version of the questionnaires to load
quest_version = 1

## Loading the pre-processed data

In [3]:
df = pd.read_csv('data/preprocessed_data_' + str(participant_number) + '_v' + str(quest_version) + '.csv')
df = df.drop(labels=['actual_day'], axis=1)

## Prepare regressor list for lazypredict

In [4]:
removed_regressors = [
    "TheilSenRegressor",
    "ARDRegression", 
    "CCA", 
    "IsotonicRegression", 
    "StackingRegressor",
    "MultiOutputRegressor", 
    "MultiTaskElasticNet", 
    "MultiTaskElasticNetCV", 
    "MultiTaskLasso", 
    "MultiTaskLassoCV", 
    "PLSCanonical", 
    "PLSRegression", 
    "RadiusNeighborsRegressor", 
    "RegressorChain", 
    "VotingRegressor", 
]

In [5]:
REGRESSORS = [
    est
    for est in all_estimators()
    if (issubclass(est[1], RegressorMixin) and (est[0] not in removed_regressors))
]

## Run LazyRegressor on dataset

### LazyRegressor for the factors influencing the sleep scores

In [6]:
# Separate the indenpendent variables from the dependent variables
y_oura = df['score']
y_sub = df['subjective_sleep_score']
X = df.drop(labels=['score', 'subjective_sleep_score'], axis=1)

# Create the test and train sets for the oura sleep score prediction
X_train_oura, X_test_oura, y_train_oura, y_test_oura = train_test_split(X, y_oura, test_size = 0.2, random_state = 26)

In [7]:
# Create the test and train sets for the subjective sleep score prediction
X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(X, y_sub, test_size = 0.2, random_state = 64)

In [8]:
# Apply the LazyRegressor on the data
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None, regressors = REGRESSORS)

#### Oura sleep scores

In [9]:
models_oura, predictions_oura = reg.fit(X_train_oura, X_test_oura, y_train_oura, y_test_oura)

print(models_oura)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:02<00:00, 17.44it/s]

                                    Adjusted R-Squared  \
Model                                                    
RANSACRegressor               266509548175091171328.00   
TransformedTargetRegressor    266509548175091171328.00   
LinearRegression              266509548175091171328.00   
Lars                                          12489.17   
GaussianProcessRegressor                         32.46   
MLPRegressor                                      2.60   
QuantileRegressor                                 1.58   
LassoLars                                         1.55   
DummyRegressor                                    1.55   
Lasso                                             1.55   
ElasticNet                                        1.28   
KNeighborsRegressor                               1.18   
LarsCV                                            1.18   
ExtraTreeRegressor                                1.12   
SVR                                               1.09   
NuSVR         

#### Subjective sleep scores

In [10]:
models_sub, predictions_sub = reg.fit(X_train_sub, X_test_sub, y_train_sub, y_test_sub)

print(models_sub)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:03<00:00, 12.93it/s]

                               Adjusted R-Squared  R-Squared   RMSE  \
Model                                                                 
Lars                                    141965.54 -273007.74 457.13   
GaussianProcessRegressor                    35.84     -66.00   7.16   
RANSACRegressor                              3.37      -3.57   1.87   
MLPRegressor                                 2.53      -1.93   1.50   
PassiveAggressiveRegressor                   2.15      -1.21   1.30   
LinearRegression                             1.89      -0.70   1.14   
TransformedTargetRegressor                   1.89      -0.70   1.14   
ExtraTreeRegressor                           1.83      -0.60   1.11   
LinearSVR                                    1.82      -0.57   1.10   
HuberRegressor                               1.74      -0.42   1.04   
DecisionTreeRegressor                        1.73      -0.40   1.04   
Ridge                                        1.70      -0.35   1.02   
Kernel

### LazyRegressor for the factors influencing the average HRV

In [11]:
# Separate the indenpendent variables from the dependent variables
y_hrv = df['average_hrv']
X_hrv = df.drop(labels=['average_hrv'], axis=1)

# Create the test and train sets for the average hrv prediction
X_train_hrv, X_test_hrv, y_train_hrv, y_test_hrv = train_test_split(X_hrv, y_hrv, test_size = 0.2, random_state = 5)

In [12]:
models_hrv, predictions_hrv = reg.fit(X_train_hrv, X_test_hrv, y_train_hrv, y_test_hrv)

print(models_hrv)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:01<00:00, 21.92it/s]

                                      Adjusted R-Squared  \
Model                                                      
LinearRegression              31062329562467961667584.00   
TransformedTargetRegressor    31062329562467961667584.00   
Lars                                              518.12   
GaussianProcessRegressor                            9.28   
MLPRegressor                                        2.74   
QuantileRegressor                                   1.55   
DummyRegressor                                      1.53   
LassoLars                                           1.48   
NuSVR                                               1.47   
SVR                                                 1.45   
ExtraTreeRegressor                                  1.27   
KNeighborsRegressor                                 1.25   
DecisionTreeRegressor                               1.22   
HuberRegressor                                      1.21   
PassiveAggressiveRegressor              